<a href="https://colab.research.google.com/github/lechurnair/TempQA/blob/main/Question%20Generation/eval.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Check the quality of the generated question using BLEU SCORE and Embedding Similarity

In [2]:
import pandas as pd
from nltk.translate.bleu_score import sentence_bleu

def calculate_bleu_score(reference, candidate):
    reference_tokens = [reference.split()]
    candidate_tokens = candidate.split()

    bleu_score = sentence_bleu(reference_tokens, candidate_tokens)
    return bleu_score

def calculate_bleu_scores(df):
    bleu_scores = []

    for index, row in df.iterrows():
        real_question = str(row['real_question'])
        predicted_question = str(row['predicted_question'])

        bleu_score = calculate_bleu_score(real_question, predicted_question)
        bleu_scores.append(bleu_score)

    df['bleu_score'] = bleu_scores

    average_bleu_score = df['bleu_score'].mean()

    return average_bleu_score, df

#The questions generated using BERT
file_path = '/content/drive/MyDrive/generated_questions_sampling.xlsx'
df = pd.read_excel(file_path)

average_bleu_score, df_with_bleu_scores = calculate_bleu_scores(df)

print(f'Average BLEU Score: {average_bleu_score}')


Average BLEU Score: 0.10038282322799107


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

In [4]:
import pandas as pd
import spacy
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import en_core_web_sm

nlp = en_core_web_sm.load()

def get_embedding(sentence):
    tokens = nlp(sentence)
    return np.mean([token.vector for token in tokens], axis=0)

def calculate_cosine_similarity(reference_sentence, predicted_sentence):
    reference_embedding = get_embedding(reference_sentence)
    predicted_embedding = get_embedding(predicted_sentence)

    reference_embedding = reference_embedding.reshape(1, -1)
    predicted_embedding = predicted_embedding.reshape(1, -1)

    similarity = cosine_similarity(reference_embedding, predicted_embedding)[0, 0]

    return similarity

file_path = '/content/drive/MyDrive/generated_questions_sampling.xlsx'
df = pd.read_excel(file_path)

cosine_similarity_scores = []
for index, row in df.iterrows():
    reference_question = str(row['real_question'])
    predicted_question = str(row['predicted_question'])

    similarity_score = calculate_cosine_similarity(reference_question, predicted_question)
    cosine_similarity_scores.append(similarity_score)

df['cosine_similarity'] = cosine_similarity_scores
print(sum(cosine_similarity_scores)/len(cosine_similarity_scores))


0.6523105222927896
